In [253]:
# REQUIREMENT for spacy
# !python -m spacy download de_core_news_sm

In [254]:
import pandas as pd
import spacy
import re
from spellchecker import SpellChecker
import os

german_spell_check = SpellChecker(language='de')
spacy_nlp = spacy.load('de_core_news_sm') 

# Data Overview

In [255]:
# self build scraper generally scrapes articles in the standard layout better, but there are also special articles which follow another layout

In [256]:
df_protocols = pd.read_csv('./data/all_bundestagsprotokolle.csv')
df_protocols.head()

,speaker,party,speech,date,applause_grüne,applause_linke,applause_fdp,applause_cdu/csu,applause_spd,applause_afd,shout_grüne,shout_linke,shout_fdp,shout_cdu/csu,shout_spd,shout_afd
0,Gabriele Katzmarek,SPD,Sie ist nicht in Stein gemeißelt – das wissen ...,2021-10-26,1,0,1,1,1,0,0,0,0,0,0,0
1,Stefan Müller,CDU/CSU,"Ich sage das deswegen, weil Demokratie ja imme...",2021-10-26,4,3,4,4,4,2,0,0,0,0,0,0
2,Britta Haßelmann,BÜNDNIS 90/DIE GRÜNEN,"Liebe Kolleginnen und Kollegen, meine Damen un...",2021-10-26,8,4,5,5,6,0,0,0,0,0,0,0
3,Marco Buschmann,FDP,"Entscheidend ist aber, dass wir heute das auss...",2021-10-26,2,2,3,2,2,1,0,0,0,0,0,0
4,Stephan Brandner,AfD,"Meine Damen und Herren, lassen Sie uns kurz ü...",2021-10-26,0,1,0,0,1,9,0,0,0,0,1,0


In [257]:
df_articles_self = pd.read_csv('./data/all_articles_self.csv')
df_articles_self.head()

,title,url,summary,last_edit_date,authors,text,topic,source,tags
0,EU und China planen Verhandlungen über möglich...,https://www.zeit.de/wirtschaft/2024-06/e-autos...,Die EU hatte China mit Strafzöllen auf E-Autos...,2024-06-22 20:20:54+02:00,['Stella Männer'],China und die EU wollen im Streit über Strafzö...,E-Autos,"Quelle: ZEIT ONLINE,\n Reuters,\n ...","['China', 'EU-Kommission', 'Aktuelle Themen']"
1,Da will ich doch wohl mal hoffen,https://www.zeit.de/2024/27/ostdeutschland-lan...,"Warum es wichtig ist, auch das Gute zu sehen. ...",2024-06-22 18:14:07+02:00,[],Neulich besuchte ich Freunde in Dresden. Wir s...,Ostdeutschland,Aus der\n ZEIT Nr. 27/2024,"['Ostdeutschland', 'Alternative für Deutschlan..."
2,Medaillenvergabe an Javier Milei im Hamburg bl...,https://www.zeit.de/politik/2024-06/argentinie...,Wenige hundert Menschen haben gegen eine Ehrun...,2024-06-22 22:30:00+02:00,['Sarah Vojta'],Hunderte Menschen haben in Hamburg gegen die V...,Argentiniens Präsident,"Quelle: ZEIT ONLINE,\n dpa","['Argentinien', 'Hamburg', 'Aktuelle Themen']"
3,Belgien siegt gegen Rumänien,https://www.zeit.de/sport/2024-06/belgien-ruma...,Belgien gewinnt nach Treffern von Tielemans un...,2024-06-22 23:00:26+02:00,[],Am zweiten Spieltag der Gruppe E hat Belgien m...,"Fußball EM, Gruppe E",NaN,"['Belgien', 'Rumänien', 'Fußball', 'Aktuelle T..."
4,Portugal steht im Achtelfinale,https://www.zeit.de/sport/2024-06/tuerkei-port...,Mit einem Sieg gegen die Türkei ist Portugal G...,2024-06-22 20:48:00+02:00,['Sarah Vojta'],Am zweiten Spieltag der Gruppe F hat Portugal ...,"Fußball-EM, Gruppe F","Quelle: ZEIT ONLINE,\n dpa","['Portugal', 'Türkei', 'Cristiano Ronaldo', 'D..."


In [258]:
df_articles_news = pd.read_csv('./data/all_articles_newspaper4k.csv')
df_articles_news.head()

,title,url,summary,last_edit_date,authors,text
0,E-Autos: EU und China planen Verhandlungen übe...,https://www.zeit.de/wirtschaft/2024-06/e-autos...,Die EU hatte China mit Strafzöllen auf E-Autos...,2024-06-22 20:20:54+02:00,"['Stella Männer', 'www.zeit.de']",China und die EU wollen im Streit über Strafzö...
1,Ostdeutschland: Da will ich doch wohl mal hoffen,https://www.zeit.de/2024/27/ostdeutschland-lan...,"Die Karten, auf denen Ostdeutschland fast komp...",2024-06-22 18:14:07+02:00,"['Valerie Schönian', 'www.zeit.de']",Neulich besuchte ich Freunde in Dresden. Wir s...
2,Argentiniens Präsident: Medaillenvergabe an Ja...,https://www.zeit.de/politik/2024-06/argentinie...,Wenige hundert Menschen haben gegen eine Ehrun...,2024-06-22 22:30:00+02:00,"['Sarah Vojta', 'www.zeit.de']",Hunderte Menschen haben in Hamburg gegen die V...
3,"Fußball EM, Gruppe E : Belgien siegt gegen Rum...",https://www.zeit.de/sport/2024-06/belgien-ruma...,Belgien gewinnt nach Treffern von Tielemans un...,2024-06-22 23:00:26+02:00,"['Melina Crispin', 'www.zeit.de']",Belgien – Rumänien 2:0 (1:0)\n\nAm zweiten Spi...
4,"Fußball-EM, Gruppe F: Portugal steht im Achtel...",https://www.zeit.de/sport/2024-06/tuerkei-port...,Mit einem Sieg gegen die Türkei ist Portugal G...,2024-06-22 20:48:00+02:00,"['Sarah Vojta', 'www.zeit.de']",Türkei – Portugal 0:3 (0:2)\n\nAm zweiten Spie...


In [259]:
df_protocols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22214 entries, 0 to 22213
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   speaker           22214 non-null  object
 1   party             22213 non-null  object
 2   speech            20107 non-null  object
 3   date              22214 non-null  object
 4   applause_grüne    22214 non-null  int64 
 5   applause_linke    22214 non-null  int64 
 6   applause_fdp      22214 non-null  int64 
 7   applause_cdu/csu  22214 non-null  int64 
 8   applause_spd      22214 non-null  int64 
 9   applause_afd      22214 non-null  int64 
 10  shout_grüne       22214 non-null  int64 
 11  shout_linke       22214 non-null  int64 
 12  shout_fdp         22214 non-null  int64 
 13  shout_cdu/csu     22214 non-null  int64 
 14  shout_spd         22214 non-null  int64 
 15  shout_afd         22214 non-null  int64 
dtypes: int64(12), object(4)
memory usage: 2.7+ MB


In [260]:
df_articles_self.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           125 non-null    object
 1   url             133 non-null    object
 2   summary         132 non-null    object
 3   last_edit_date  133 non-null    object
 4   authors         133 non-null    object
 5   text            131 non-null    object
 6   topic           125 non-null    object
 7   source          77 non-null     object
 8   tags            133 non-null    object
dtypes: object(9)
memory usage: 9.5+ KB


In [261]:
df_articles_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           135 non-null    object
 1   url             135 non-null    object
 2   summary         135 non-null    object
 3   last_edit_date  135 non-null    object
 4   authors         135 non-null    object
 5   text            135 non-null    object
dtypes: object(6)
memory usage: 6.5+ KB


In [262]:
df_protocols.describe()

,applause_grüne,applause_linke,applause_fdp,applause_cdu/csu,applause_spd,applause_afd,shout_grüne,shout_linke,shout_fdp,shout_cdu/csu,shout_spd,shout_afd
count,22214.000000,22214.000000,22214.000000,22214.000000,22214.000000,22214.000000,22214.000000,22214.000000,22214.000000,22214.000000,22214.000000,22214.000000
mean,2.738903,0.588503,2.248042,1.786216,2.792248,0.959845,0.088503,0.016386,0.040650,0.101423,0.102773,0.074683
std,3.209514,1.641145,2.949486,2.739190,3.223207,1.933181,0.436794,0.177270,0.251076,0.505251,0.509327,0.436954
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,0.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,0.000000,3.000000,3.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,136.000000,22.000000,133.000000,52.000000,136.000000,23.000000,12.000000,10.000000,6.000000,13.000000,13.000000,12.000000


# Data Cleaning

In [263]:
# merge newspaper columns into self columns where values are missing
# lowercase texts
# remove non alphabetic chars and special characters in text and html encodings e.g. &#34; for 34
# perform stop word removal
# perform lemmatization on text

In [264]:
def remove_non_alphabetic_chars(text):
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w ]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower() # we lose information, because in german every noun is capitalized -> nevertheless can be helpful, especially with a small dataset

def correct_word(word):
    corrected = german_spell_check.correction(word)
    return corrected if corrected else word

def perform_spell_check(text):
    doc = spacy_nlp(text)
    return ' '.join([correct_word(token.text) for token in doc if not token.is_stop]) # Tokenization, Stop_Word_Removal and Spellcheck

def clean_text(text, spell_check = True):
    text = remove_non_alphabetic_chars(text)
    # after performing the spell check it can correct non stop_words to stop_words (eaine -> eine) and we need to check for stop_words again
    doc = spacy_nlp(perform_spell_check(text) if spell_check else text) 
    cleaned_text = ' '.join([token.lemma_.lower() for token in doc if not token.is_stop]) # Lemmatization: Reduce words to their base form (not root)
    return cleaned_text

def filter_list(sentences: list[str], words_to_remove: list[str]) -> list[str]:
    filtered_list = []
    for sentence in sentences:
        include_string = True
        for word in words_to_remove:
            if word in sentence:
                include_string = False
                break
        if include_string:
            filtered_list.append(sentence)

    return filtered_list

In [265]:
# clean articles
articles_cleaned_file = "./data/all_articles_cleaned.csv"
if not os.path.exists(articles_cleaned_file):
    df_articles_self.set_index('url', inplace=True)
    df_articles_news.set_index('url', inplace=True)
    
    df_articles_cleaned = df_articles_self.copy()
    df_articles_cleaned = df_articles_cleaned.dropna(subset=['text']) # missing texts -> premium article or media article
    # Fill NaN values in df_articles_cleaned with corresponding values from df_articles_news
    df_articles_cleaned['title'] = df_articles_cleaned['title'].fillna(df_articles_news['title'])
    df_articles_cleaned['last_edit_date'] = df_articles_cleaned['last_edit_date'].fillna(df_articles_news['last_edit_date'])
    df_articles_cleaned['summary'] = df_articles_cleaned['summary'].fillna(df_articles_news['summary'])
    df_articles_cleaned['tags'] = df_articles_cleaned['tags'].fillna("unbekannt")
    df_articles_cleaned['source'] = df_articles_cleaned['source'].fillna("unbekannt")
    df_articles_cleaned['topic'] = df_articles_cleaned['topic'].fillna("unbekannt")

    author_filter = ["zeit", "index"]
    df_articles_cleaned['authors'] = df_articles_cleaned['authors'].apply(eval)
    df_articles_cleaned['authors'] = df_articles_cleaned['authors'].apply(lambda x: filter_list(x, author_filter))

    df_articles_cleaned[['text', 'summary', 'title', 'topic']] = df_articles_cleaned[['text', 'summary', 'title', 'topic']].map(lambda x: clean_text(x, False))
    df_articles_cleaned.reset_index().to_csv(articles_cleaned_file, index=False)
else:
    df_articles_cleaned = pd.read_csv(articles_cleaned_file)

In [267]:
# clean protocols
protocols_cleaned_file = "./data/all_protocols_cleaned.csv"
if not os.path.exists(protocols_cleaned_file):
    df_protocols_cleaned = df_protocols.copy()
    df_protocols_cleaned = df_protocols_cleaned.dropna(subset=['speech', 'party'])
    df_protocols_cleaned['speech'] = df_protocols_cleaned['speech'].apply(lambda x: clean_text(x, False))
    df_protocols_cleaned = df_protocols_cleaned.drop_duplicates(subset=['speech'])
    df_protocols_cleaned.to_csv(protocols_cleaned_file, index=False)
else:
    df_protocols_cleaned = pd.read_csv(protocols_cleaned_file)

# Data Analysis

In [ ]:
# Top 10 Redner mit dem meisten Applaus
# Top 10 Reden mit dem meisten Applaus
# Parteien Applausverteilung
# Parteien Interruptionsverteilung
# Wordcloud speeches
# Redenverteilung
# Reden Word Count Verteilung
# Reden Word Count Verteilung by party

In [ ]:
# Top 10 Article Topics
# Wordcloud articles text
# Wordcloud article title
# Text Word Count Verteilung

# Models

In [ ]:
# Predict claps für rede, regression

In [ ]:
# Was will ich machen? 
# Partei zugehörigkeit eines Texts predicten von Bundestags reden
# Sentiment eines Artikels/ einer Rede predicten